In [1]:
import sys
sys.path.append('/content/drive/Othercomputers/내 iMac/Meta_Learning/코스웍/2022 고급기계학습주제 (김광인)/과제/CW1')

In [2]:
from utils.utils import *
from algorithms.random_forest import RandomForest

In [3]:
import numpy as np
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import accuracy_score

In [4]:
# seed fixation
np.random.seed(1)

In [5]:
# path for colab only
import os
colab = '/content/drive/Othercomputers/내 iMac/Meta_Learning/코스웍/2022 고급기계학습주제 (김광인)/과제/CW1/data'
PATH1 = os.path.join(colab, 'cifar-10-batches-py/data_batch_1')
PATH2 = os.path.join(colab, 'cifar-100-python/train')

# Load data
cifar10 = unpickle(PATH1)
cifar100 = unpickle(PATH2)

# Generate dataset
cifar10 = dataset_generator(cifar10, 3000, cifar_type='cifar10')
cifar100 = dataset_generator(cifar100, 3000, cifar_type='cifar100')

# test path
PATH1_t = os.path.join(colab, 'cifar-10-batches-py/data_batch_2')
PATH2_t = os.path.join(colab, 'cifar-100-python/test')

# Load test data
test_cifar10 = unpickle(PATH1_t)
test_cifar100 = unpickle(PATH2_t)

# Generate test dataset
test_cifar10 = dataset_generator(test_cifar10, 300, cifar_type='cifar10')
test_cifar100 = dataset_generator(test_cifar100, 300, cifar_type='cifar100')

# Normalise
cifar10['data'] = cifar10['data']/255
cifar100['data'] = cifar100['data']/255

test_cifar10['data'] = test_cifar10['data']/255
test_cifar100['data'] = test_cifar100['data']/255

In [6]:
X10 = cifar10['data']
y10 = np.array(cifar10['labels'])
X100 = cifar100['data']
y100 = np.array(cifar100['labels'])

test_X10 = test_cifar10['data']
test_y10 = np.array(test_cifar10['labels'])
test_X100 = test_cifar100['data']
test_y100 = np.array(test_cifar100['labels'])

## Hyperparameter Search: Train/Test Set
According to DecisionTree analysis,<br>
__cifar10 best model__: {'max_depth': 100, 'max_features': 55, 'min_sample_split': 100}, accuracy: __0.28__<br>

In the case of cifar100, since training time requires a lot in best model, I choose the second-best model for this project.<br>
__cifar100 best model__: {'max_depth': 100, 'max_features': 200, 'min_sample_split': 30}, accuracy: __0.063__<br>
__cifar100 second-best model__:{'max_depth': 100, 'max_features': 100, 'min_sample_split': 50}, accuracy: __0.06__

### cifar10

In [ ]:
param_grid = {
    'max_depth': [100],
    'min_sample_split': [100],
    'max_features': [55],
    'n_trees': [3, 5, 10]
}

grid = list(ParameterGrid(param_grid))

In [ ]:
grid

[{'max_depth': 100, 'max_features': 55, 'min_sample_split': 100, 'n_trees': 3},
 {'max_depth': 100, 'max_features': 55, 'min_sample_split': 100, 'n_trees': 5},
 {'max_depth': 100,
  'max_features': 55,
  'min_sample_split': 100,
  'n_trees': 10}]

In [ ]:
import time

train_time = []
inf_time = []
acc = []

for i, args in enumerate(grid):
    print(args)
    clf = RandomForest(**args)
    
    # check time
    start = time.time()
    
    print(f'{i}th classifier training...', end='')
    clf.fit(X10, y10)
    
    end1 = time.time()
    train_time.append(end1-start)
    print( 'time: ', round(end1-start,1), end='')

    y_pred = clf.predict(test_X10)
    ac = accuracy_score(test_y10, y_pred)
    print(' acc: ', ac)

    end2 = time.time()
    inf_time.append(end2-end1)
    acc.append(ac)

# Rounding
tt = [round(t,1) for t in train_time]
it = [round(t,4) for t in inf_time]
ac = [round(a,3) for a in acc]

trial = {'train_time': tt, 'inference_time': it, 'accuracy': ac}

import json

with open("RF_cifar10.json", "w") as json_file:
    json.dump(trial, json_file)

print()
for args, a in zip(grid, ac):
    print(args, a)

{'max_depth': 100, 'max_features': 55, 'min_sample_split': 100, 'n_trees': 3}
0th classifier training...time:  483.8 acc:  0.26
{'max_depth': 100, 'max_features': 55, 'min_sample_split': 100, 'n_trees': 5}
1th classifier training...time:  878.9 acc:  0.29
{'max_depth': 100, 'max_features': 55, 'min_sample_split': 100, 'n_trees': 10}
2th classifier training...time:  1668.2 acc:  0.3333333333333333

{'max_depth': 100, 'max_features': 55, 'min_sample_split': 100, 'n_trees': 3} 0.26
{'max_depth': 100, 'max_features': 55, 'min_sample_split': 100, 'n_trees': 5} 0.29
{'max_depth': 100, 'max_features': 55, 'min_sample_split': 100, 'n_trees': 10} 0.333


In [ ]:
print(trial)

{'train_time': [483.8, 878.9, 1668.2], 'inference_time': [0.0074, 0.0099, 0.0177], 'accuracy': [0.26, 0.29, 0.333]}


## cifar100

In [ ]:
param_grid = {
    'max_depth': [100],
    'min_sample_split': [150],
    'max_features': [100],
    'n_trees': [3, 5, 10]
}

grid = list(ParameterGrid(param_grid))

In [ ]:
grid

[{'max_depth': 100,
  'max_features': 100,
  'min_sample_split': 150,
  'n_trees': 3},
 {'max_depth': 100,
  'max_features': 100,
  'min_sample_split': 150,
  'n_trees': 5},
 {'max_depth': 100,
  'max_features': 100,
  'min_sample_split': 150,
  'n_trees': 10}]

In [ ]:
import time

train_time = []
inf_time = []
acc = []

for i, args in enumerate(grid):
    print(args)
    clf = RandomForest(**args)
    
    # check time
    start = time.time()
    
    print(f'{i}th classifier training...', end='')
    clf.fit(X100, y100)
    
    end1 = time.time()
    train_time.append(end1-start)
    print( 'time: ', round(end1-start,1), end='')

    y_pred = clf.predict(test_X100)
    ac = accuracy_score(test_y100, y_pred)
    print(' acc: ', ac)

    end2 = time.time()
    inf_time.append(end2-end1)
    acc.append(ac)

# Rounding
tt = [round(t,1) for t in train_time]
it = [round(t,4) for t in inf_time]
ac = [round(a,3) for a in acc]

trial_ = {'train_time': tt, 'inference_time': it, 'accuracy': ac}

import json

with open("RF_cifar100.json", "w") as json_file:
    json.dump(trial_, json_file)

print()
for args, a in zip(grid, ac):
    print(args, a)

print()
print(trial_)

{'max_depth': 100, 'max_features': 100, 'min_sample_split': 150, 'n_trees': 3}
0th classifier training...time:  965.1 acc:  0.03666666666666667
{'max_depth': 100, 'max_features': 100, 'min_sample_split': 150, 'n_trees': 5}
1th classifier training...time:  1606.5 acc:  0.043333333333333335
{'max_depth': 100, 'max_features': 100, 'min_sample_split': 150, 'n_trees': 10}
2th classifier training...time:  3248.9 acc:  0.02666666666666667

{'max_depth': 100, 'max_features': 100, 'min_sample_split': 150, 'n_trees': 3} 0.037
{'max_depth': 100, 'max_features': 100, 'min_sample_split': 150, 'n_trees': 5} 0.043
{'max_depth': 100, 'max_features': 100, 'min_sample_split': 150, 'n_trees': 10} 0.027

{'train_time': [965.1, 1606.5, 3248.9], 'inference_time': [0.0066, 0.0088, 0.0129], 'accuracy': [0.037, 0.043, 0.027]}


## Hyperparameter Search: Cross-validation

In [10]:
from sklearn.model_selection import StratifiedKFold

seed = 1
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

### cifar10

In [ ]:
param_grid = {
    'max_depth': [100],
    'min_sample_split': [100],
    'max_features': [55],
    'n_trees': [3, 5, 10]
}

grid = list(ParameterGrid(param_grid))

In [ ]:
grid

[{'max_depth': 100, 'max_features': 55, 'min_sample_split': 100, 'n_trees': 3},
 {'max_depth': 100, 'max_features': 55, 'min_sample_split': 100, 'n_trees': 5},
 {'max_depth': 100,
  'max_features': 55,
  'min_sample_split': 100,
  'n_trees': 10}]

In [ ]:
import time
from copy import deepcopy

_time = []
acc = {'mean_acc':[], 'std_acc':[]}

best_acc = -1

for i, args in enumerate(grid):
    print(f'{i}th classifier training...')
    clf = RandomForest(**args)
    
    sub_acc = []
    start = time.time()
    for train_index, test_index in skf.split(X10, y10):
        X_train, X_test = X10[train_index], X10[test_index]
        y_train, y_test = y10[train_index], y10[test_index]

        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        
        ac = accuracy_score(y_test, y_pred)
        sub_acc.append(ac)

    end = time.time()
    
    print('time: ', round(end-start,1),end='')
    _time.append(end-start)
    acc['mean_acc'].append(np.mean(sub_acc))
    acc['std_acc'].append(np.std(sub_acc))
    print(' acc: ', acc['mean_acc'][i], end='')
    print(' std: ', acc['std_acc'][i])
    
    # best model
    if np.mean(sub_acc) > best_acc:
        best_acc = np.mean(sub_acc)
        best_clf = deepcopy(clf)

# Rounding
tt = [round(t,1) for t in _time]
ac1 = [round(a,3) for a in acc['mean_acc']]
ac2 = [round(a,3) for a in acc['std_acc']]

trial_cv = {'time': tt, 'mean_accuracy': ac1, 'std_accuracy': ac2}

print()
print(trial_cv)

import json

with open("RF_cifar10_cv.json", "w") as json_file:
    json.dump(trial_cv, json_file)


##########################
# SAVE-LOAD using pickle #
##########################
import pickle

# save
with open('random_forest_model.pkl','wb') as f:
    pickle.dump(best_clf,f)

0th classifier training...
time:  1766.0 acc:  0.22766666666666668 std:  0.009695359714832659
1th classifier training...
time:  3013.9 acc:  0.26766666666666666 std:  0.008339997335464538
2th classifier training...


In [ ]:
import time
from copy import deepcopy

_time = []
acc = {'mean_acc':[], 'std_acc':[]}

best_acc = -1

for i, args in enumerate(grid[2:]):
    print(f'{i+2}th classifier training...')
    clf = RandomForest(**args)
    
    sub_acc = []
    start = time.time()
    for train_index, test_index in skf.split(X10, y10):
        X_train, X_test = X10[train_index], X10[test_index]
        y_train, y_test = y10[train_index], y10[test_index]

        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        
        ac = accuracy_score(y_test, y_pred)
        sub_acc.append(ac)

    end = time.time()
    
    print('time: ', round(end-start,1),end='')
    _time.append(end-start)
    acc['mean_acc'].append(np.mean(sub_acc))
    acc['std_acc'].append(np.std(sub_acc))
    print(' acc: ', acc['mean_acc'][i], end='')
    print(' std: ', acc['std_acc'][i])
    
    # best model
    if np.mean(sub_acc) > best_acc:
        best_acc = np.mean(sub_acc)
        best_clf = deepcopy(clf)

# Rounding
tt = [round(t,1) for t in _time]
ac1 = [round(a,3) for a in acc['mean_acc']]
ac2 = [round(a,3) for a in acc['std_acc']]

trial_cv = {'time': tt, 'mean_accuracy': ac1, 'std_accuracy': ac2}

print()
print(trial_cv)

2th classifier training...
time:  7016.2 acc:  0.29566666666666663 std:  0.010519822558706333

{'time': [7016.2], 'mean_accuracy': [0.296], 'std_accuracy': [0.011]}


### cifar100

In [7]:
param_grid = {
    'max_depth': [100],
    'min_sample_split': [150],
    'max_features': [100],
    'n_trees': [3, 5, 10]
}

grid = list(ParameterGrid(param_grid))

In [8]:
grid

[{'max_depth': 100,
  'max_features': 100,
  'min_sample_split': 150,
  'n_trees': 3},
 {'max_depth': 100,
  'max_features': 100,
  'min_sample_split': 150,
  'n_trees': 5},
 {'max_depth': 100,
  'max_features': 100,
  'min_sample_split': 150,
  'n_trees': 10}]

In [ ]:
import time
from copy import deepcopy

_time = []
acc = {'mean_acc':[], 'std_acc':[]}

best_acc = -1

for i, args in enumerate(grid):
    print(f'{i}th classifier training...')
    clf = RandomForest(**args)
    
    sub_acc = []
    start = time.time()
    for train_index, test_index in skf.split(X100, y100):
        X_train, X_test = X100[train_index], X100[test_index]
        y_train, y_test = y100[train_index], y100[test_index]

        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        
        ac = accuracy_score(y_test, y_pred)
        sub_acc.append(ac)

    end = time.time()
    
    print('time: ', round(end-start,1),end='')
    _time.append(end-start)
    acc['mean_acc'].append(np.mean(sub_acc))
    acc['std_acc'].append(np.std(sub_acc))
    print(' acc: ', acc['mean_acc'][i], end='')
    print(' std: ', acc['std_acc'][i])
    
    # best model
    if np.mean(sub_acc) > best_acc:
        best_acc = np.mean(sub_acc)
        best_clf = deepcopy(clf)

# Rounding
tt = [round(t,1) for t in _time]
ac1 = [round(a,3) for a in acc['mean_acc']]
ac2 = [round(a,3) for a in acc['std_acc']]

trial_cv_ = {'time': tt, 'mean_accuracy': ac1, 'std_accuracy': ac2}

print()
print(trial_cv_)

import json

with open("RF_cifar100_cv.json", "w") as json_file:
    json.dump(trial_cv_, json_file)


##########################
# SAVE-LOAD using pickle #
##########################
import pickle

# save
with open('random_forest_model_cifar100.pkl','wb') as f:
    pickle.dump(best_clf,f)

0th classifier training...
time:  4210.0 acc:  0.039 std:  0.0042946995755750415
1th classifier training...


KeyboardInterrupt: ignored

In [ ]:
import time
from copy import deepcopy

_time = []
acc = {'mean_acc':[], 'std_acc':[]}

best_acc = -1

for i, args in enumerate(grid[1:]):
    print(f'{i+1}th classifier training...')
    clf = RandomForest(**args)
    
    sub_acc = []
    start = time.time()
    for train_index, test_index in skf.split(X100, y100):
        X_train, X_test = X100[train_index], X100[test_index]
        y_train, y_test = y100[train_index], y100[test_index]

        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        
        ac = accuracy_score(y_test, y_pred)
        sub_acc.append(ac)

    end = time.time()
    
    print('time: ', round(end-start,1),end='')
    _time.append(end-start)
    acc['mean_acc'].append(np.mean(sub_acc))
    acc['std_acc'].append(np.std(sub_acc))
    print(' acc: ', acc['mean_acc'][i], end='')
    print(' std: ', acc['std_acc'][i])
    
    # best model
    if np.mean(sub_acc) > best_acc:
        best_acc = np.mean(sub_acc)
        best_clf = deepcopy(clf)

# Rounding
tt = [round(t,1) for t in _time]
ac1 = [round(a,3) for a in acc['mean_acc']]
ac2 = [round(a,3) for a in acc['std_acc']]

trial_cv_ = {'time': tt, 'mean_accuracy': ac1, 'std_accuracy': ac2}

print()
print(trial_cv_)

1th classifier training...
time:  6963.4 acc:  0.044000000000000004 std:  0.007644896627453143
2th classifier training...


In [11]:
import time
from copy import deepcopy

_time = []
acc = {'mean_acc':[], 'std_acc':[]}

best_acc = -1

for i, args in enumerate(grid[2:]):
    print(f'{i+2}th classifier training...')
    clf = RandomForest(**args)
    
    sub_acc = []
    start = time.time()
    for train_index, test_index in skf.split(X100, y100):
        X_train, X_test = X100[train_index], X100[test_index]
        y_train, y_test = y100[train_index], y100[test_index]

        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        
        ac = accuracy_score(y_test, y_pred)
        sub_acc.append(ac)

    end = time.time()
    
    print('time: ', round(end-start,1),end='')
    _time.append(end-start)
    acc['mean_acc'].append(np.mean(sub_acc))
    acc['std_acc'].append(np.std(sub_acc))
    print(' acc: ', acc['mean_acc'][i], end='')
    print(' std: ', acc['std_acc'][i])
    
    # best model
    if np.mean(sub_acc) > best_acc:
        best_acc = np.mean(sub_acc)
        best_clf = deepcopy(clf)

# Rounding
tt = [round(t,1) for t in _time]
ac1 = [round(a,3) for a in acc['mean_acc']]
ac2 = [round(a,3) for a in acc['std_acc']]

trial_cv_ = {'time': tt, 'mean_accuracy': ac1, 'std_accuracy': ac2}

print()
print(trial_cv_)

2th classifier training...
time:  14451.7 acc:  0.046666666666666676 std:  0.0045946829173634076

{'time': [14451.7], 'mean_accuracy': [0.047], 'std_accuracy': [0.005]}
